# Introduction

python document的解释：A weak reference to an object is not enough to keep the object alive: when the only remaining references to a referent are weak references, garbage collection is free to destroy the referent and reuse its memory for something else. However, until the object is actually destroyed the weak reference may return the object even if there are no strong references to it.

也就是说，当一个对象仅有一个weak reference指向它的时候，python的gc可以随意地销毁该对象。

## 1. weakref.WeakValueDictionary, WeakKeyDictionary, WeakSet
因此，weak reference 的主要用处是实现cache，即当一个对象不用的时候，cache里可以不保存这个对象。为了方便weak reference cache的实现，`weakref`模块提供了几个比较方便的collection，如：`WeakKeyDictionary`, `WeakValueDictionary`, `WeakSet`。

举个例子：

In [12]:
class BigData:
    pass
    
d = dict()

def test():
    b1 = BigData()
    d["b1"] = b1
    b2 = BigData()
    d["b2"] = b2
    print(len(d)) # 2

test()
print(len(d)) # 2

2
2


In [11]:
from weakref import WeakValueDictionary

class BigData:
    pass
    
d = WeakValueDictionary()

def test():
    b1 = BigData()
    d["b1"] = b1
    b2 = BigData()
    d["b2"] = b2
    print(len(d)) # 2
    # gc销毁b1和b2

test()
print(len(d)) # 2

2
0


## 2. weakref.ref
可以发现，对于一个普通的dict，就算没有变量引用dict中的value，这个value也不会被gc销毁，因为普通dict里对于对象的引用不是weak reference。而对于`WeakValueDictionary`，一旦没有变量引用字典中的value的时候，gc就会把字典中的key-value pair销毁。

一般来说，用这几个collection足够支撑大部分的应用场景了，不过`weakref`模板也提供了一些low-level的方法，用户可以自己创建一个对象的weak reference，通过`weakref.ref(object[, callback])`方法。

例如下面例子，`weakref.ref`创建一个ref对象。当它reference的对象活着的时候，调用r()返回它指向的对象；当对象b被销毁的时候，r的callback方法被调用，且调用r()返回None：

In [45]:
import weakref

class BigData:
    def __init__(self, value):
        self.value = value

def test():
    b = BigData(3)
    r = weakref.ref(b, lambda ref: print(f"{ref} destroyed"))
    print(r().value)
    del b
    print(r())

test()

3
<weakref at 0x0000021923379F98; dead> destroyed
None


## 3. weakref.finalize

finalize的主要好处是可以很方便地为一个对象的销毁创建一个callback函数（不用保存finalize的返回值，ref必须保存返回值）。

比如下面例子，

In [3]:
import weakref

class BigData:
    def __init__(self, value):
        self.value = value

def test_ref():
    b = BigData(3)
    weakref.ref(b, lambda ref: print(f"destroyed in test_ref"))
    del b

def test_finalize():
    b = BigData(3)
    weakref.finalize(b, lambda: print(f"destroyed in test_finalize"))
    del b
    
test_ref()
test_finalize()

destroyed in test_finalize


finalizer还有一个好处是可以用来实现资源回收，例如：

In [57]:
import weakref
import tempfile
import shutil

def rmtree(name):
    print(f"removing {name}..")
    shutil.rmtree(name)
    
class TempDir:
    def __init__(self):
        self.name = tempfile.mkdtemp()
        self._finalizer = weakref.finalize(self, rmtree, self.name)

    def remove(self):
        self._finalizer()

    @property
    def removed(self):
        return not self._finalizer.alive

        
def test():
    temp = TempDir()
    print(temp.name)
    
test()

C:\Users\zhipwang\AppData\Local\Temp\2\tmp_juwqrhz
removing C:\Users\zhipwang\AppData\Local\Temp\2\tmp_juwqrhz..
